In [2]:
import pandas as pd
import tensorflow as tf
import numpy
target = pd.read_csv('train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id').apply(lambda x: x-1)
df_train_values_original = pd.read_csv('train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'uint8',
    'geo_level_3_id':'uint8', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')


pd.options.display.float_format = '{:20,.2f}'.format

In [3]:
df = df_train_values_original.drop(columns=["geo_level_2_id", "geo_level_3_id"])

In [4]:
def one_hot_encode_data(dataframe, column_name):
    dummies = pd.get_dummies(dataframe[column_name])
    rename_columns = {}
    for column in dummies.columns.values:
        rename_columns[column] = column_name + '_' + column
    return dataframe.drop(columns=[column_name]).join(dummies.rename(columns=rename_columns))

In [5]:
df = one_hot_encode_data(df,'land_surface_condition')
df = one_hot_encode_data(df,'foundation_type')
df = one_hot_encode_data(df,'roof_type')
df = one_hot_encode_data(df,'ground_floor_type')
df = one_hot_encode_data(df,'other_floor_type')
df = one_hot_encode_data(df,'position')
df = one_hot_encode_data(df,'plan_configuration')
df = one_hot_encode_data(df,'legal_ownership_status')
df = one_hot_encode_data(df,'geo_level_1_id')
df

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,geo_level_1_id_28,geo_level_1_id_29,geo_level_1_id_3,geo_level_1_id_30,geo_level_1_id_4,geo_level_1_id_5,geo_level_1_id_6,geo_level_1_id_7,geo_level_1_id_8,geo_level_1_id_9
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
28830,2,10,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
94947,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
590882,2,10,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201944,3,30,8,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,1,55,6,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
669485,2,0,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
602512,3,55,6,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
dataset = tf.data.Dataset.from_tensor_slices((df[:int(len(df)/1)].values, target[:int(len(df)/1)].values))
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))


Features: [ 2 30  6  5  1  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  1  0  0  1  0  0  1  0  0  1  0  0  0  0  0  1  0  0  0
  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0], Target: [2]
Features: [ 2 10  8  7  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  1  0  0  0  1  0  0  1  0  0  0  0  0  1  0  0  1  0  0  0
  0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0], Target: [1]
Features: [ 2 10  5  5  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  1  0  0  1  0  0  1  0  0  1  0  0  0  0  0  0  0  1  0
  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0], Target: [2]
Features: [ 2 10  6  5  0  1  0  0  0  0  1  1  0  0

In [7]:
def compile_model(l_size, l_amount, activation):
    layers = []
    for x in range(l_amount):
        layers.append(tf.keras.layers.Dense(l_size, activation=activation))
    layers.append(tf.keras.layers.Dense(units=3, activation='softmax'))
    model = tf.keras.Sequential(layers)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(optimizer='Adagrad',
                  loss=loss_fn,
                  metrics=['accuracy'])
    return model

In [ ]:
grid_search = [
    [384, 480],
    [6, 7],
    ['relu',]
]
results = []
with tf.device('/CPU:0'):
    for n in grid_search[1]:
        for i in grid_search[0]:
            for a in grid_search[2]:
                m = compile_model(i, n, a)
                m.fit(dataset.batch(32), epochs=50)
                results.append((i, n, a, m.evaluate(df.to_numpy(), target.to_numpy())[1]))

In [ ]:
for line in results:
    print(line)

In [ ]:
j = 0
for n in grid_search[1]:
    for i in grid_search[0]:
        for a in grid_search[2]:
            print(results[j][1])
            j+=1
for n in grid_search[1]:
    for i in grid_search[0]:
        for a in grid_search[2]:
            print(str(n)+','+str(i)+','+str(a))

In [ ]:
results[0][1]

In [8]:
best_model = compile_model(480, 6, 'relu')
best_model.fit(dataset.batch(32), epochs=1)

8144/8144 [==============================] - 36s 4ms/step - loss: 0.7902 - accuracy: 0.6143


In [9]:
best_model.fit(dataset.batch(32), epochs=400)

Epoch 1/400
8144/8144 [==============================] - 35s 4ms/step - loss: 0.7121 - accuracy: 0.6626
Epoch 2/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.7008 - accuracy: 0.6690
Epoch 3/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6960 - accuracy: 0.6715
Epoch 4/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6933 - accuracy: 0.6729
Epoch 5/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6914 - accuracy: 0.6740
Epoch 6/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6900 - accuracy: 0.6746
Epoch 7/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6889 - accuracy: 0.6753
Epoch 8/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6880 - accuracy: 0.6760
Epoch 9/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6872 - accuracy: 0.6764
Epoch 10/400
8144/8144 [==============================] - 34s 4m

8144/8144 [==============================] - 34s 4ms/step - loss: 0.6638 - accuracy: 0.6917
Epoch 80/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6636 - accuracy: 0.6919
Epoch 81/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6633 - accuracy: 0.6921
Epoch 82/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6631 - accuracy: 0.6921
Epoch 83/400
8144/8144 [==============================] - 35s 4ms/step - loss: 0.6628 - accuracy: 0.6923
Epoch 84/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6626 - accuracy: 0.6924
Epoch 85/400
8144/8144 [==============================] - 35s 4ms/step - loss: 0.6624 - accuracy: 0.6926
Epoch 86/400
8144/8144 [==============================] - 35s 4ms/step - loss: 0.6621 - accuracy: 0.6927
Epoch 87/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6619 - accuracy: 0.6930
Epoch 88/400
8144/8144 [==============================] - 34s 4ms/st

8144/8144 [==============================] - 33s 4ms/step - loss: 0.6338 - accuracy: 0.7095
Epoch 234/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6336 - accuracy: 0.7095
Epoch 235/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6334 - accuracy: 0.7096
Epoch 236/400
8144/8144 [==============================] - 33s 4ms/step - loss: 0.6332 - accuracy: 0.7097
Epoch 237/400
8144/8144 [==============================] - 33s 4ms/step - loss: 0.6331 - accuracy: 0.7098
Epoch 238/400
8144/8144 [==============================] - 33s 4ms/step - loss: 0.6329 - accuracy: 0.7099
Epoch 239/400
8144/8144 [==============================] - 33s 4ms/step - loss: 0.6327 - accuracy: 0.7100
Epoch 240/400
8144/8144 [==============================] - 33s 4ms/step - loss: 0.6325 - accuracy: 0.7102
Epoch 241/400
8144/8144 [==============================] - 34s 4ms/step - loss: 0.6324 - accuracy: 0.7102
Epoch 242/400
8144/8144 [==============================] - 3

8144/8144 [==============================] - 36s 4ms/step - loss: 0.6081 - accuracy: 0.7234
Epoch 388/400
8144/8144 [==============================] - 36s 4ms/step - loss: 0.6079 - accuracy: 0.7236
Epoch 389/400
8144/8144 [==============================] - 37s 5ms/step - loss: 0.6078 - accuracy: 0.7236
Epoch 390/400
8144/8144 [==============================] - 38s 5ms/step - loss: 0.6076 - accuracy: 0.7236
Epoch 391/400
8144/8144 [==============================] - 41s 5ms/step - loss: 0.6075 - accuracy: 0.7238
Epoch 392/400
8144/8144 [==============================] - 40s 5ms/step - loss: 0.6073 - accuracy: 0.7238
Epoch 393/400
8144/8144 [==============================] - 39s 5ms/step - loss: 0.6072 - accuracy: 0.7240
Epoch 394/400
8144/8144 [==============================] - 40s 5ms/step - loss: 0.6070 - accuracy: 0.7241
Epoch 395/400
8144/8144 [==============================] - 43s 5ms/step - loss: 0.6069 - accuracy: 0.7241
Epoch 396/400
8144/8144 [==============================] - 4

In [10]:
best_model.save('modelos/NNModelGS2')

INFO:tensorflow:Assets written to: modelos/NNModelGS2\assets
